In [1]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, asin, acos, atan
%reload_ext autoreload
%autoreload 2
plt.style.use('latex.mplstyle')

from point_configuration import PointConfiguration

In [5]:
from algorithms import reconstruct_mds
from algorithms import reconstruct_srls
from algorithms import reconstruct_acd
from algorithms import reconstruct_sdp
from algorithms import reconstruct_dwmds
import sys

def localization_error(point_estimate):
    err = np.linalg.norm(p.points[:n] - point_estimate[:n])**2/n
    return err

def try_method(errs_dict, failed, method):
    try: 
        xhat = execute_method(method)
        err = localization_error(xhat)
        errs_dict[method][i].append(err)
        return xhat
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise
        failed.append(method)
        
def execute_method(method):
    if method == 'MDS':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, method='geometric')
    
    if method == 'MDSoptspace':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, 
                                   method='geometric', mask=weights, 
                                   completion='optspace', print_out=True)
    if method == 'MDSalternate':
        xhat = reconstruct_mds(noisy_edm, real_points=p.points, 
                                   method='geometric', mask=weights, 
                                   completion='alternate', print_out=True)
    if method == 'SDR':
        x_SDRold, EDMbest = reconstruct_sdp(noisy_edm, weights, lamda, p.points)
        ### Added to avoid strange "too large to be a matrix" error
        xhat = np.zeros((N, d))
        xhat[:,:] = x_SDRold
    if method == 'ACD':
        xhat, fs, err_edms, err_points = reconstruct_acd(noisy_edm, weights, X_0=x0.copy(), 
                                                          real_points=p.points)
    if method == 'dwMDS':
        xhat, costs = reconstruct_dwmds(noisy_edm, X=x0.copy(), W=weights, n=n)
    if method == 'SRLS':
        xhat = reconstruct_srls(noisy_edm, p.points, indices=range(n))
    return xhat
    
def create_weights(N, proportion=0.5, method='all'):
    # create indices object to choose from
    if method == 'all':
        all_indices = np.triu_indices(N,1)
    elif method == 'first':
        all_indices = [np.zeros(N-1).astype(np.int), np.arange(1,N).astype(np.int)]
    ntotal = len(all_indices[0])
    nchosen = int(ntotal * proportion)
    
    # randomly choose from indices and set to 0
    choice = np.random.choice(ntotal, nchosen, replace=False)
    chosen = [all_indices[0][choice], all_indices[1][choice]]
    weights = np.ones((N,N))
    weights[chosen] = 0
    weights[chosen[1],chosen[0]] = 0
    weights[range(N), range(N)] = 0
    return weights
    
print(create_weights(9, method='first'))
print(create_weights(9, method='all'))

[[ 0.  0.  1.  1.  0.  0.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  0.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  0.  1.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  0.  1.  1.  1.  1.]
 [ 0.  1.  1.  1.  1.  0.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  0.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  0.  1.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.  0.]]
[[ 0.  1.  0.  1.  1.  1.  0.  1.  0.]
 [ 1.  0.  1.  0.  1.  1.  1.  0.  1.]
 [ 0.  1.  0.  0.  1.  0.  0.  1.  1.]
 [ 1.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 1.  1.  1.  0.  0.  1.  0.  0.  0.]
 [ 1.  1.  0.  1.  1.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  1.  0.  0.  0.  0.  1.]
 [ 0.  1.  1.  0.  0.  0.  0.  1.  0.]]


In [4]:
import copy
#from plots_cti import plot_point_sets_3d, plot_matrix
from basics import create_noisy_edm, get_edm

N = 10
#m = 9
m = 9
d = 3
lamda = 1000
square_size = 5
noises = np.logspace(-4,-1,10) 
#noises = np.logspace(-2,0,20) #missing higherror
max_it = 30
x0 = np.zeros((N, d)) # set by method chosen below. 
method_x0 = 'SDR'

errs = [[] for noise in noises]
errs_dict = {
    'noise':copy.deepcopy(errs),
    'MDS':copy.deepcopy(errs),
    'MDSalternate':copy.deepcopy(errs),
    'MDSoptspace':copy.deepcopy(errs)}
    #'SDR':copy.deepcopy(errs),
    #'ACD':copy.deepcopy(errs),
    #'dwMDS':copy.deepcopy(errs),
    #'SRLS':copy.deepcopy(errs)}

n = N - m
n_edges = (N*(N-1))/2
p = PointConfiguration(N, d)

for it in range(max_it):
    print(it)
    p.set_points('random',size=square_size)
    
    for i, noise in enumerate(noises):
        # missing distances
        weights = create_weights(N, method='first')
    
        failed = []
        print(noise)
        noisy_edm = create_noisy_edm(p.edm, noise)
        err_noise = np.linalg.norm(p.edm - noisy_edm)**2/n_edges
        errs_dict['noise'][i].append(err_noise)

        for method in errs_dict.keys():
            if method != 'noise':
                if method == method_x0:
                    x0 = try_method(errs_dict, failed, method)
                else:
                    try_method(errs_dict, failed, method)
        
        if len(failed)>0:
            print('exception occured in',failed)
        else:
            for method in errs_dict.keys():
                print('error {}: {}'.format(method, errs_dict[method][i][-1]))
np.save('results/missing_errs_dict_n{}_m{}.npy'.format(n,m),errs_dict)

0
0.0001
alternate: relative error:0.01235360841426369
optspace: relative error:3.2137260719670754e-05
error noise: 4.062905368280107e-08
error MDS: 1.9376273608865653e-10
error MDSalternate: 0.06918702099743514
error MDSoptspace: 0.000252842621088097
0.000215443469003
alternate: relative error:0.00046078724162755026
optspace: relative error:0.05139173227983705
error noise: 2.152308598024258e-07
error MDS: 1.9510575305847973e-09
error MDSalternate: 0.009472999917826404
error MDSoptspace: 1.0209559799261052
0.000464158883361
alternate: relative error:0.00033577281140130733
optspace: relative error:5.714857807131656e-06
error noise: 1.0137505032710504e-06
error MDS: 7.642892247164434e-09
error MDSalternate: 0.0076023751422147
error MDSoptspace: 4.269471473174428e-05
0.001
alternate: relative error:0.003681263453877635
optspace: relative error:0.03493790024231455
error noise: 3.8027401817120214e-06
error MDS: 4.162481416890636e-08
error MDSalternate: 0.09614444520575478
error MDSoptspace: